In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

import pandas as pd
import random
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 

In [3]:
# 창 열기
driver = webdriver.Chrome()

## 주방

### 1) 식탁 테이블

In [4]:
# 사이트 열기
driver.get('https://ohou.se/store/category?category_id=10270018')
driver.implicitly_wait(10)
time.sleep(3)

# 리뷰많은순으로 변경
pre_order = driver.find_element(By.CLASS_NAME, "category-filter-bar-order-button").click()
time.sleep(0.5+random.random())

order_ = driver.find_elements(By.CLASS_NAME, "property-filter-panel-entry-list__item__title")
order_[-3].click()
time.sleep(0.5+random.random())

In [5]:
info = []

ffffa = 0
# 스크롤 내리기
def scroll():

    global ffffa
    ffffa += 600 
    driver.execute_script("window.scrollTo(0, "+ str(ffffa) + ")")

# 제목 : [브랜드, 제목, 가격, 이미지위치]
# 스크롤을 조금씩 내리면서 가져오기 때문에 데이터 중복 가능
# 데이터 중복 제거하기 위해 제목은 중복안되니까 제목을 key로 지정 
data = {}

# 스크롤 조금씩 내리면서 데이터 가져오기
for page in tqdm(range(20)):
    scroll()
    time.sleep(5)

    # 브랜드, 상품명, 가격, 이미지 경로, 제품 링크
    brands =  driver.find_elements(By.CLASS_NAME, "production-item__header__brand")
    names =  driver.find_elements(By.CLASS_NAME, "production-item__header__name")
    prices = driver.find_elements(By.CLASS_NAME, "production-item-price__price")
    srcs=  driver.find_elements(By.CLASS_NAME, "image")
    links =  driver.find_elements(By.CLASS_NAME, "production-item__overlay")

    amount= len(brands)
    print(amount ,"개 데이터 가져옴....")
    
    for i in range(amount):
        # 브랜드
        brand = brands[i].text

        # 상품명
        name = names[i].text
        name = re.sub(r'\[.*?\]', '', name)

        # 가격
        price = prices[i].text

        # 이미지 경로
        src = srcs[i].get_attribute('src')

        # 제품 링크
        link = links[i].get_attribute("href")

        data[name] = [brand, name, price, src, link]


  0%|          | 0/20 [00:00<?, ?it/s]

15 개 데이터 가져옴....


  5%|▌         | 1/20 [00:05<01:43,  5.45s/it]

21 개 데이터 가져옴....


 10%|█         | 2/20 [00:11<01:39,  5.53s/it]

...

 90%|█████████ | 18/20 [01:41<00:11,  5.66s/it]

24 개 데이터 가져옴....


 95%|█████████▌| 19/20 [01:46<00:05,  5.65s/it]

27 개 데이터 가져옴....


100%|██████████| 20/20 [01:52<00:00,  5.62s/it]


In [6]:
result_amount = len(data.keys())
result_amount

102

In [7]:
i = 0
info = []

for key,value in tqdm(data.items()):
    try:
        brand = value[0]
        name = value[1]
        price = value[2]
        src = value[3]
        link = value[4]

        i += 1
        time.sleep(0.5 + random.random())
        response = requests.get(src, stream=True)
        print(src)

        save_image_path = f'./static/img/kitchen/table/table{i}.png'
        if response.status_code == 200:
            # 응답 내용을 바이너리 파일로 저장하기
            with open(save_image_path, 'wb') as out_file:
                out_file.write(response.content)
        
        info.append([brand, name, price, save_image_path, link, None])

    except:
        print('ERROR!!')
        pass

  1%|          | 1/102 [00:01<01:49,  1.08s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/1557909311160_Hw9mBQUpIR.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


  2%|▏         | 2/102 [00:02<02:08,  1.29s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/165838354735796647.png?gif=1&w=360&h=360&c=c&q=0.8&webp=1


  3%|▎         | 3/102 [00:03<01:46,  1.08s/it]

...

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/168955604020927106.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


 99%|█████████▉| 101/102 [02:36<00:02,  2.00s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/168750944442185707.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


100%|██████████| 102/102 [02:37<00:00,  1.55s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/167693602190744947.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


In [8]:
len(info)

99

In [9]:
# 제품 링크 접속하기
for i in tqdm(range(len(info))):
    info_line = info[i]
    link = info_line[4]

    driver.get(link)
    driver.implicitly_wait(4)
    time.sleep(0.5+random.random())

    re_ = driver.find_elements(By.CLASS_NAME, "production-selling-navigation__item")
    re_[1].click()
    time.sleep(0.5+random.random()) 

    # 한 페이지당 5개 리뷰 
    # 20페이지 크롤링해서 총 100개 리뷰 가져오기
    review = []
    first = driver.find_elements(By.CLASS_NAME, "production-review-item__description")
    for i in first:
        review.append(i.text)

    for i in range(19):
        try:
            driver.find_elements(By.CLASS_NAME, "_2XI47._3I7ex")[0].click()
            time.sleep(1.5+random.random())
            text = driver.find_elements(By.CLASS_NAME, "production-review-item__description")
            for i in text:
                review.append(i.text)
        except:
            print('ERROR!!')
            pass

    info_line[5] = review



100%|██████████| 99/99 [1:16:35<00:00, 46.42s/it]


In [10]:
dining_table = pd.DataFrame(info, columns=['brand', 'item', 'price', 'image', 'link', 'review'])
dining_table

,brand,item,price,image,link,review
0,프리메이드,원형 티 카페 테이블 2size,"39,900",./static/img/kitchen/table/table1.png,https://ohou.se/productions/123785/selling?aff...,[몇달 사용하고 올립니다~ 싸이즈도 혼자 또는 둘이 밥먹고 술마시기 너무 좋고 튼튼...
1,오트밀하우스,로지 원형 카페 테이블 800 3colors,"78,900",./static/img/kitchen/table/table2.png,https://ohou.se/productions/1241971/selling?af...,[이 가격에 이 정도 품질이라니 놀랍네요. 꺼떡거림 없이 안정적이라 팔을 올려두고 ...
2,비알프렌드,디디테이블 반타원 라미네이트 화이트식탁 1000(2size),"97,580",./static/img/kitchen/table/table3.png,https://ohou.se/productions/933994/selling?aff...,[딸 자취방에 선물로 사줬답니다 바닥에 상펴고 먹을꺼 생각하니 신경쓰여서 구입해줬어...
3,비알프렌드,디디테이블 반타원 라미네이트 화이트식탁 1200(2size),"109,880",./static/img/kitchen/table/table4.png,https://ohou.se/productions/934051/selling?aff...,[조리대에 딱맞는 테이블이라 구매했어요! 이전엔 5센티 넓어서 툭튀였는데 이젠 딱맞...
4,오트밀하우스,로지 원형 카페 테이블 700 3colors,"72,900",./static/img/kitchen/table/table5.png,https://ohou.se/productions/1241968/selling?af...,"[구매해서 잘쓰고있습니당 !추천드립니다, 그냥 흰색을 살까 고심하다가 골랐는데 적당..."
...,...,...,...,...,...,...
94,소프시스,올리브 접이식 테이블 1000x600,"45,900",./static/img/kitchen/table/table98.png,https://ohou.se/productions/1742932/selling?af...,[바닥에 앉아서 낮은테이블만 쓰다가 소파에 앉아 높은 테이블 쓰니싸 편하고 좋네요 ...
95,폴앤코코,플랜비 실버 타원형 식탁 테이블 1600 화이트 다이닝,"198,000",./static/img/kitchen/table/table99.png,https://ohou.se/productions/1416023/selling?af...,"[배송 왕 빠르고, 식탁도 오셔서 잘 조립해주셨습니다! 마감처리도 너무 훌륭하고, ..."
96,모노폴리오,헤이 매거진랙 반타원 식탁 화이트 책상 카페 테이블 1000 2colors,"114,900",./static/img/kitchen/table/table100.png,https://ohou.se/productions/2125262/selling?af...,[몇주 전 수령 후 이제 조립했는데 생각했던것 만큼 예쁘고 실용적입니다 여성이라면 ...
97,폴인퍼니,베네치아 4인용 12T 타원 포세린 세라믹 식탁테이블 세트 1600,"299,000",./static/img/kitchen/table/table101.png,https://ohou.se/productions/871179/selling?aff...,[딱 기본 느낌의 화이트 식탁이에요. 세라믹 식탁이라 좀 시끄럽긴 한데 잘 닦이고 ...


In [11]:
dining_table.to_csv('dining_table.csv')

---

### 2) 식탁의자

In [12]:
# 사이트 열기
driver.get('https://ohou.se/store/category?category_id=10270021')
driver.implicitly_wait(10)
time.sleep(3)

# 리뷰많은순으로 변경
pre_order = driver.find_element(By.CLASS_NAME, "category-filter-bar-order-button").click()
time.sleep(0.5+random.random())

order_ = driver.find_elements(By.CLASS_NAME, "property-filter-panel-entry-list__item__title")
order_[-3].click()
time.sleep(0.5+random.random())

In [13]:
info = []

ffffa = 0
# 스크롤 내리기
def scroll():

    global ffffa
    ffffa += 600 
    driver.execute_script("window.scrollTo(0, "+ str(ffffa) + ")")

# 제목 : [브랜드, 제목, 가격, 이미지위치]
# 스크롤을 조금씩 내리면서 가져오기 때문에 데이터 중복 가능
# 데이터 중복 제거하기 위해 제목은 중복안되니까 제목을 key로 지정 
data = {}

# 스크롤 조금씩 내리면서 데이터 가져오기
for page in tqdm(range(20)):
    scroll()
    time.sleep(5)

    # 브랜드, 상품명, 가격, 이미지 경로, 제품 링크
    brands =  driver.find_elements(By.CLASS_NAME, "production-item__header__brand")
    names =  driver.find_elements(By.CLASS_NAME, "production-item__header__name")
    prices = driver.find_elements(By.CLASS_NAME, "production-item-price__price")
    srcs=  driver.find_elements(By.CLASS_NAME, "image")
    links =  driver.find_elements(By.CLASS_NAME, "production-item__overlay")

    amount= len(brands)
    print(amount ,"개 데이터 가져옴....")
    
    for i in range(amount):
        # 브랜드
        brand = brands[i].text

        # 상품명
        name = names[i].text
        name = re.sub(r'\[.*?\]', '', name)

        # 가격
        price = prices[i].text

        # 이미지 경로
        src = srcs[i].get_attribute('src')

        # 제품 링크
        link = links[i].get_attribute("href")

        data[name] = [brand, name, price, src, link]


  0%|          | 0/20 [00:00<?, ?it/s]

15 개 데이터 가져옴....


  5%|▌         | 1/20 [00:06<01:56,  6.14s/it]

21 개 데이터 가져옴....


 10%|█         | 2/20 [00:12<01:54,  6.39s/it]

...

 90%|█████████ | 18/20 [02:00<00:13,  6.81s/it]

27 개 데이터 가져옴....


 95%|█████████▌| 19/20 [02:07<00:06,  6.84s/it]

24 개 데이터 가져옴....


100%|██████████| 20/20 [02:13<00:00,  6.69s/it]


In [14]:
result_amount = len(data.keys())
result_amount

96

In [15]:
i = 0
info = []

for key,value in tqdm(data.items()):
    try:
        brand = value[0]
        name = value[1]
        price = value[2]
        src = value[3]
        link = value[4]

        i += 1
        time.sleep(0.5 + random.random())
        response = requests.get(src, stream=True)
        print(src)

        save_image_path = f'./static/img/kitchen/chair/chair{i}.png'
        if response.status_code == 200:
            # 응답 내용을 바이너리 파일로 저장하기
            with open(save_image_path, 'wb') as out_file:
                out_file.write(response.content)
        
        info.append([brand, name, price, save_image_path, link, None])

    except:
        print('ERROR!!')
        pass

  1%|          | 1/96 [00:01<02:44,  1.73s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/164325182439107818.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


  2%|▏         | 2/96 [00:02<02:11,  1.40s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/162748750075449137.jpeg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


  3%|▎         | 3/96 [00:03<01:48,  1.17s/it]

...

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/168117318543679779.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


 99%|█████████▉| 95/96 [02:42<00:01,  1.81s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/168551339956958842.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


100%|██████████| 96/96 [02:43<00:00,  1.70s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/162738839999955418.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


In [16]:
len(info)

93

In [17]:
# 제품 링크 접속하기
# 회원만 구입할 수 있는 상품이 포함되어 있어 로그인 후 크롤링
for i in tqdm(range(len(info))):
    info_line = info[i]
    link = info_line[4]

    driver.get(link)
    driver.implicitly_wait(4)
    time.sleep(0.5+random.random())

    re_ = driver.find_elements(By.CLASS_NAME, "production-selling-navigation__item")
    re_[1].click()
    time.sleep(0.5+random.random()) 

    # 한 페이지당 5개 리뷰 
    # 20페이지 크롤링해서 총 100개 리뷰 가져오기
    review = []
    first = driver.find_elements(By.CLASS_NAME, "production-review-item__description")
    for i in first:
        review.append(i.text)

    for i in range(19):
        try:
            driver.find_elements(By.CLASS_NAME, "_2XI47._3I7ex")[0].click()
            time.sleep(1.5+random.random())
            text = driver.find_elements(By.CLASS_NAME, "production-review-item__description")
            for i in text:
                review.append(i.text)
        except:
            print('ERROR!!')
            pass

    info_line[5] = review



100%|██████████| 93/93 [1:16:49<00:00, 49.56s/it]


In [18]:
chair = pd.DataFrame(info, columns=['brand', 'item', 'price', 'image', 'link', 'review'])
chair

,brand,item,price,image,link,review
0,영가구,베가 폴라 투명 접이식의자 17colors 대량구매추가할인,"25,600",./static/img/kitchen/chair/chair1.png,https://ohou.se/productions/354736/selling?aff...,[자취방에 의자가 없었어서 옛날부터 사고싶던 투명의자 드디어 샀는데 이뻐요 집 꾸미...
1,프리메이드,에펠의자 고급형 4colors,"29,900",./static/img/kitchen/chair/chair2.png,https://ohou.se/productions/125457/selling?aff...,"[이사와서 베란다에 배치했는데, 사이즈도 의자 무게도 적당해서 너무 좋아요. 약하면..."
2,끌레오,위니 식탁 의자 인테리어 포인트 카페 체어 SL084,"14,600",./static/img/kitchen/chair/chair3.png,https://ohou.se/productions/201149/selling?aff...,[이사진 찾으려고 인스타 스토리 뒤졌네요ㅋㅋㅋ인테리어 넘 좋아하지만 감각은 없는1인...
3,로제까사,국민 에펠체어 7colors,"19,500",./static/img/kitchen/chair/chair4.png,https://ohou.se/productions/24381/selling?affe...,"[저는 화장대 의자로 쓰고 있어요 ㅎㅎ 좋습니당! 3년째 튼튼, 사람들이 에펠체어를..."
4,에코상사,감성 인테리어 원목 원목 미니 원형스툴,"19,900",./static/img/kitchen/chair/chair5.png,https://ohou.se/productions/343687/selling?aff...,[아기가 이제 집을 열심히 돌아다녀서 안전 고려해서 구매했어요. 아무 생각없이 상부...
...,...,...,...,...,...,...
88,보니애가구,비올렛 디자인체어 2colors,"58,700",./static/img/kitchen/chair/chair92.png,https://ohou.se/productions/49198/selling?affe...,"[편하고 고급스럽습니다. 완제품이라 더 맘에 들어요., 앉으면 폭,길이 다 작은 느..."
89,로포텐,빙글빙글 원목 화장대의자 회전 화장대스툴 의자,"29,800",./static/img/kitchen/chair/chair93.png,https://ohou.se/productions/182021/selling?aff...,[가격대비 너무 좋아용 바닥 끌리는 소리도 잘 안나기도하고 귀엽고 사용하는 화장대랑...
90,인블루가구,메탈 스툴 보조 화장대의자 부클레 디자인 체어 8colors,"42,000",./static/img/kitchen/chair/chair94.png,https://ohou.se/productions/1060390/selling?af...,[생각보다 검수가 별로 안되어 오네요. 의자 바닥면에 표시위해 써놓은 분필? 같은 ...
91,피카소가구,(고급형) 체코 디자인 원목 카페 식탁 의자 3colors,"59,500",./static/img/kitchen/chair/chair95.png,https://ohou.se/productions/279977/selling?aff...,[식탁의자 바꿨는데 상품평들이 너무 좋아서 구매했습니다. 식탁이랑 완전세트 입니다....


In [19]:
chair.to_csv('chair.csv')

---

### 3) 렌지대

In [20]:
# 사이트 열기
driver.get('https://ohou.se/store/category?category_id=10270022')
driver.implicitly_wait(10)
time.sleep(3)

# 리뷰많은순으로 변경
pre_order = driver.find_element(By.CLASS_NAME, "category-filter-bar-order-button").click()
time.sleep(0.5+random.random())

order_ = driver.find_elements(By.CLASS_NAME, "property-filter-panel-entry-list__item__title")
order_[-3].click()
time.sleep(0.5+random.random())

In [21]:
info = []

ffffa = 0
# 스크롤 내리기
def scroll():

    global ffffa
    ffffa += 600 
    driver.execute_script("window.scrollTo(0, "+ str(ffffa) + ")")

# 제목 : [브랜드, 제목, 가격, 이미지위치]
# 스크롤을 조금씩 내리면서 가져오기 때문에 데이터 중복 가능
# 데이터 중복 제거하기 위해 제목은 중복안되니까 제목을 key로 지정 
data = {}

# 스크롤 조금씩 내리면서 데이터 가져오기
for page in tqdm(range(20)):
    scroll()
    time.sleep(5)

    # 브랜드, 상품명, 가격, 이미지 경로, 제품 링크
    brands =  driver.find_elements(By.CLASS_NAME, "production-item__header__brand")
    names =  driver.find_elements(By.CLASS_NAME, "production-item__header__name")
    prices = driver.find_elements(By.CLASS_NAME, "production-item-price__price")
    srcs=  driver.find_elements(By.CLASS_NAME, "image")
    links =  driver.find_elements(By.CLASS_NAME, "production-item__overlay")

    amount= len(brands)
    print(amount ,"개 데이터 가져옴....")
    
    for i in range(amount):
        # 브랜드
        brand = brands[i].text

        # 상품명
        name = names[i].text
        name = re.sub(r'\[.*?\]', '', name)

        # 가격
        price = prices[i].text

        # 이미지 경로
        src = srcs[i].get_attribute('src')

        # 제품 링크
        link = links[i].get_attribute("href")

        data[name] = [brand, name, price, src, link]


  0%|          | 0/20 [00:00<?, ?it/s]

15 개 데이터 가져옴....


  5%|▌         | 1/20 [00:06<01:57,  6.20s/it]

21 개 데이터 가져옴....


 10%|█         | 2/20 [00:12<01:55,  6.41s/it]

...

 90%|█████████ | 18/20 [02:03<00:14,  7.18s/it]

27 개 데이터 가져옴....


 95%|█████████▌| 19/20 [02:10<00:07,  7.30s/it]

24 개 데이터 가져옴....


100%|██████████| 20/20 [02:18<00:00,  6.92s/it]


In [22]:
result_amount = len(data.keys())
result_amount

105

In [23]:
i = 0
info = []

for key,value in tqdm(data.items()):
    try:
        brand = value[0]
        name = value[1]
        price = value[2]
        src = value[3]
        link = value[4]

        i += 1
        time.sleep(0.5 + random.random())
        response = requests.get(src, stream=True)
        print(src)

        save_image_path = f'./static/img/kitchen/oven_stand/ovst{i}.png'
        if response.status_code == 200:
            # 응답 내용을 바이너리 파일로 저장하기
            with open(save_image_path, 'wb') as out_file:
                out_file.write(response.content)
        
        info.append([brand, name, price, save_image_path, link, None])

    except:
        print('ERROR!!')
        pass

  1%|          | 1/105 [00:01<03:16,  1.89s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/165478976553242384.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


  2%|▏         | 2/105 [00:03<03:14,  1.89s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/169452302913657911.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


  3%|▎         | 3/105 [00:05<03:26,  2.02s/it]

...

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/162944033153300120.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1
https://image.ohou.se/i/bucketplace-v2-development/uploads%2Fproductions%2F1486088535911_b1.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


 99%|█████████▉| 104/105 [03:12<00:02,  2.24s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/162674406530078967.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


100%|██████████| 105/105 [03:14<00:00,  1.85s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/openapi/814499/1667974874552.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


In [24]:
len(info)

105

In [25]:
# 제품 링크 접속하기
# 회원만 구입할 수 있는 상품이 포함되어 있어 로그인 후 크롤링
for i in tqdm(range(len(info))):
    info_line = info[i]
    link = info_line[4]

    driver.get(link)
    driver.implicitly_wait(4)
    time.sleep(0.5+random.random())

    re_ = driver.find_elements(By.CLASS_NAME, "production-selling-navigation__item")
    re_[1].click()
    time.sleep(0.5+random.random()) 

    # 한 페이지당 5개 리뷰 
    # 20페이지 크롤링해서 총 100개 리뷰 가져오기
    review = []
    first = driver.find_elements(By.CLASS_NAME, "production-review-item__description")
    for i in first:
        review.append(i.text)

    for i in range(19):
        try:
            driver.find_elements(By.CLASS_NAME, "_2XI47._3I7ex")[0].click()
            time.sleep(1.5+random.random())
            text = driver.find_elements(By.CLASS_NAME, "production-review-item__description")
            for i in text:
                review.append(i.text)
        except:
            print('ERROR!!')
            pass

    info_line[5] = review



100%|██████████| 105/105 [1:30:10<00:00, 51.53s/it]

ERROR!!


In [26]:
oven_stand = pd.DataFrame(info, columns=['brand', 'item', 'price', 'image', 'link', 'review'])
oven_stand

,brand,item,price,image,link,review
0,가구레시피,국내생산 조립식 시그니처 진열장형 3단 전자렌지대 다용도수납장 밥솥다이,"27,100",./static/img/kitchen/oven_stand/ovst1.png,https://ohou.se/productions/258147/selling?aff...,[전자레인지 놓기에 딱이에요! 주방 공간이 협소해서 전자레인지 따로 놓을 거치대로 ...
1,가구레시피,국내생산 조립식 시그니처 진열장형 4단 전자렌지대 다용도수납장 밥솥다이,"36,900",./static/img/kitchen/oven_stand/ovst2.png,https://ohou.se/productions/388291/selling?aff...,[이사하고 짐 정리 덜됫을때여서 지져분한점 양해해주세용!! 전자레인지 에어프라이기 ...
2,가구레시피,국내생산 조립식 시그니처 렌지대형 4단 전자렌지대 다용도수납장 주방선반,"47,600",./static/img/kitchen/oven_stand/ovst3.png,https://ohou.se/productions/388334/selling?aff...,"[원래 주방수납장 칸 높이가 높지 않아 밥솥만 따로 스툴 위에 놓았었는데, 그게 불..."
3,가구레시피,국내생산 조립식 시그니처 렌지대형 3단 전자렌지대 다용도수납장 주방선반,"37,600",./static/img/kitchen/oven_stand/ovst4.png,https://ohou.se/productions/388323/selling?aff...,[기존엔 렌지와 밥솥 거치대로 사용했다가 옵션 있는 집으로 이사하게 되었는데도 수납...
4,블루밍홈,잭 스틸렌지대 3colors,"32,900",./static/img/kitchen/oven_stand/ovst5.png,https://ohou.se/productions/48363/selling?affe...,"[하나 사고 하나 더 사서 아주 잘 쓰고 있습니다, 튼튼하고 사이즈도 너무 좋고 집..."
...,...,...,...,...,...,...
100,바로방가구,시크 전자레인지 수납 렌지대 6colors,"34,900",./static/img/kitchen/oven_stand/ovst101.png,https://ohou.se/productions/390654/selling?aff...,"[저자리 다 들어내고 설치했는데 만족!, 싸제불량품질 배보다 배꼽이더비싼 배송비, ..."
101,하우스팡,높이조절 넓이조절 전자레인지 선반 3단,"36,700",./static/img/kitchen/oven_stand/ovst102.png,https://ohou.se/productions/980290/selling?aff...,[1. 길이 조절하기 마음에 듭니다 2. 색상 블랙 마음에 쏙 듭니다 3. 튼튼합니...
102,데코라인,시크 렌지대 겸 식탁세트 오리지널,"129,000",./static/img/kitchen/oven_stand/ovst103.png,https://ohou.se/productions/23476/selling?affe...,[깔끔하고 설치도 잘 맞읏우요 재가 사용하는 렌지랑은 사이즈가 안맞아수 아쉬웠지만 ...
103,나무숲,베이크 800 틈새장 전자렌지수납장 렌지대 밥솥수납장 3colors,"51,900",./static/img/kitchen/oven_stand/ovst104.png,https://ohou.se/productions/933408/selling?aff...,"[사이즈 딱 맞게 구매했어요! 가격 저렴하고 좋네요^^, 크기가 완전 딱맞아서 기부..."


In [27]:
oven_stand.to_csv('oven_stand.csv')

---

### 4) 수납장

In [28]:
# 사이트 열기
driver.get('https://ohou.se/store/category?category_id=10270052')
driver.implicitly_wait(10)
time.sleep(3)

# 리뷰많은순으로 변경
pre_order = driver.find_element(By.CLASS_NAME, "category-filter-bar-order-button").click()
time.sleep(0.5+random.random())

order_ = driver.find_elements(By.CLASS_NAME, "property-filter-panel-entry-list__item__title")
order_[-3].click()
time.sleep(0.5+random.random())

In [29]:
info = []

ffffa = 0
# 스크롤 내리기
def scroll():

    global ffffa
    ffffa += 600 
    driver.execute_script("window.scrollTo(0, "+ str(ffffa) + ")")

# 제목 : [브랜드, 제목, 가격, 이미지위치]
# 스크롤을 조금씩 내리면서 가져오기 때문에 데이터 중복 가능
# 데이터 중복 제거하기 위해 제목은 중복안되니까 제목을 key로 지정 
data = {}

# 스크롤 조금씩 내리면서 데이터 가져오기
for page in tqdm(range(20)):
    scroll()
    time.sleep(5)

    # 브랜드, 상품명, 가격, 이미지 경로, 제품 링크
    brands =  driver.find_elements(By.CLASS_NAME, "production-item__header__brand")
    names =  driver.find_elements(By.CLASS_NAME, "production-item__header__name")
    prices = driver.find_elements(By.CLASS_NAME, "production-item-price__price")
    srcs=  driver.find_elements(By.CLASS_NAME, "image")
    links =  driver.find_elements(By.CLASS_NAME, "production-item__overlay")

    amount= len(brands)
    print(amount ,"개 데이터 가져옴....")
    
    for i in range(amount):
        # 브랜드
        brand = brands[i].text

        # 상품명
        name = names[i].text
        name = re.sub(r'\[.*?\]', '', name)

        # 가격
        price = prices[i].text

        # 이미지 경로
        src = srcs[i].get_attribute('src')

        # 제품 링크
        link = links[i].get_attribute("href")

        data[name] = [brand, name, price, src, link]


  0%|          | 0/20 [00:00<?, ?it/s]

15 개 데이터 가져옴....


  5%|▌         | 1/20 [00:05<01:44,  5.50s/it]

21 개 데이터 가져옴....


 10%|█         | 2/20 [00:11<01:39,  5.54s/it]

...

 90%|█████████ | 18/20 [01:41<00:11,  5.66s/it]

27 개 데이터 가져옴....


 95%|█████████▌| 19/20 [01:47<00:05,  5.68s/it]

27 개 데이터 가져옴....


100%|██████████| 20/20 [01:53<00:00,  5.65s/it]


In [30]:
result_amount = len(data.keys())
result_amount

108

In [31]:
i = 0
info = []

for key,value in tqdm(data.items()):
    try:
        brand = value[0]
        name = value[1]
        price = value[2]
        src = value[3]
        link = value[4]

        i += 1
        time.sleep(0.5 + random.random())
        response = requests.get(src, stream=True)
        print(src)

        save_image_path = f'./static/img/kitchen/cabinet/cab{i}.png'
        if response.status_code == 200:
            # 응답 내용을 바이너리 파일로 저장하기
            with open(save_image_path, 'wb') as out_file:
                out_file.write(response.content)
        
        info.append([brand, name, price, save_image_path, link, None])

    except:
        print('ERROR!!')
        pass

  1%|          | 1/108 [00:01<03:16,  1.83s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/168006784570630681.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


  2%|▏         | 2/108 [00:02<02:29,  1.41s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/166259430577377615.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


  3%|▎         | 3/108 [00:04<02:54,  1.66s/it]

...

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/160082643866658797.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


 99%|█████████▉| 107/108 [02:52<00:01,  1.99s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/1560826133017_ol6nVgE.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


100%|██████████| 108/108 [02:55<00:00,  1.62s/it]

https://image.ohou.se/i/bucketplace-v2-development/uploads/productions/160197307210080038.jpg?gif=1&w=360&h=360&c=c&q=0.8&webp=1


In [32]:
len(info)

105

In [33]:
# 제품 링크 접속하기
# 회원만 구입할 수 있는 상품이 포함되어 있어 로그인 후 크롤링
for i in tqdm(range(len(info))):
    info_line = info[i]
    link = info_line[4]

    driver.get(link)
    driver.implicitly_wait(4)
    time.sleep(0.5+random.random())

    re_ = driver.find_elements(By.CLASS_NAME, "production-selling-navigation__item")
    re_[1].click()
    time.sleep(0.5+random.random()) 

    # 한 페이지당 5개 리뷰 
    # 20페이지 크롤링해서 총 100개 리뷰 가져오기
    review = []
    first = driver.find_elements(By.CLASS_NAME, "production-review-item__description")
    for i in first:
        review.append(i.text)

    for i in range(19):
        try:
            driver.find_elements(By.CLASS_NAME, "_2XI47._3I7ex")[0].click()
            time.sleep(1.5+random.random())
            text = driver.find_elements(By.CLASS_NAME, "production-review-item__description")
            for i in text:
                review.append(i.text)
        except:
            print('ERROR!!')
            pass

    info_line[5] = review



 27%|██▋       | 28/105 [21:35<1:02:48, 48.94s/it]

ERROR!!


 28%|██▊       | 29/105 [22:28<1:03:24, 50.05s/it]

ERROR!!
ERROR!!


 29%|██▊       | 30/105 [23:19<1:03:09, 50.52s/it]

...

ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!


 99%|█████████▉| 104/105 [1:49:56<01:20, 80.40s/it]

ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!
ERROR!!


100%|██████████| 105/105 [1:51:14<00:00, 63.57s/it]

ERROR!!


In [34]:
cabinet = pd.DataFrame(info, columns=['brand', 'item', 'price', 'image', 'link', 'review'])
cabinet

,brand,item,price,image,link,review
0,아리아퍼니쳐,Tribeca 주방/거실 높은 수납장 1000+상부장추가선택,"159,000",./static/img/kitchen/cabinet/cab1.png,https://ohou.se/productions/452975/selling?aff...,[디자인 너무 깔끔하고 서랍도 부드럽게 잘 열리고 닫히고 수납도 넉넉해서 완전 맘에...
1,필웰,RZ9757 데이아 서랍형 이동조리대 DVX,"89,000",./static/img/kitchen/cabinet/cab2.png,https://ohou.se/productions/390565/selling?aff...,[새가구 사서 기분좋게 사용하고 싶었는데 포장뜯기전부터 기분이 안좋네요 리뷰 둘러보...
2,필웰,RZ9492 데이아 이동조리대400 DVX,"59,000",./static/img/kitchen/cabinet/cab3.png,https://ohou.se/productions/390566/selling?aff...,[나무색이 때도안타고 좋음 흰색부분은 때탐 이동식이라 좋아요 판도 두껍고 단단해요!...
3,필웰,ST9839 무노 주방 서랍수납장800 DVX,"109,000",./static/img/kitchen/cabinet/cab4.png,https://ohou.se/productions/413790/selling?aff...,[위에는 화장품 넣고 아래는 옷장으로 쓰기도 했고 지금은 위에는 잠옷 아래는 바지 ...
4,리센트,파스타 주방수납장,"159,000",./static/img/kitchen/cabinet/cab5.png,https://ohou.se/productions/300318/selling?aff...,"[크기도적당하고 예쁘네요 특히 수납도 알차서 좋아요, 원하는 날짜에 배송해주셨고 바..."
...,...,...,...,...,...,...
100,리센트,퐁듀 속깊은 주방수납장 1000,"169,000",./static/img/kitchen/cabinet/cab104.png,https://ohou.se/productions/1455446/selling?af...,[주방 수납장과 샹활용품 담아놓기에 넓고 좋아요 정수기도 올려놓으니 좁은 주방이 활...
101,장인가구,셰리 그레이 주방수납장 8종,"93,000",./static/img/kitchen/cabinet/cab105.png,https://ohou.se/productions/39689/selling?affe...,[가격대비 괜찬아요 근데 좀 낮은 느낌.. 다리를 따로 구입해서 달아줫더니 예뻐요~...
102,DF동서가구,허브 600 주방수납장(소),"44,900",./static/img/kitchen/cabinet/cab106.png,https://ohou.se/productions/484965/selling?aff...,"[생각보다 배송도 빨랐구 튼튼하고 좋네요, 진작 살 걸 하는 맘이었어요. 깔끔하게 ..."
103,한샘,머그 아일랜드 주방수납장,"267,000",./static/img/kitchen/cabinet/cab107.png,https://ohou.se/productions/130453/selling?aff...,[기사님이 빠르고 설치해주고 가셨습니다! 서랍도 생각보다 커서 수납도 좋고 디자인도...


In [35]:
cabinet.to_csv('cabinet.csv')